In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from twitter.twitter_collection import TwitterCollection
from data_pipelines.data_manipulation import DataManipulation
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = 'ignore', category = pd.errors.PerformanceWarning)
%matplotlib inline

Python-dotenv could not parse statement starting at line 11


In [2]:
from collections import Counter
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('stopwords')
from nltk.corpus import stopwords   # python -m nltk.downloader stopwords // from commandline

stop = stopwords.words('english')
import matplotlib.pyplot as plt
import pandas as pd
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import re

[nltk_data] Downloading package stopwords to /home/kozan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
client_twitter = TwitterCollection()
df_btc = client_twitter.get_tweets_df('bitcoin', pathdf='../data/tweets_data/btc/', filedf='btc_tweets.csv')

Authentication Successfull


In [4]:
df_btc.head()

,created_at,text,source,name,username,location,verified,description
0,2022-11-24 09:19:09+00:00,If you got your #btc #eth stuck in #blo...,Twitter Web App,Okay Bears Sales,Virgini21645863,NaN,False,Free tag to ONLY moots.
1,2022-11-24 09:19:08+00:00,@KuWallet Download #KuCoinWallet and bet and w...,Twitter Web App,Albalooo,Albaloookhoshke,"so far , so close",False,We all have something that make us really happ...
2,2022-11-24 09:19:06+00:00,The current data for #deXit \n \nprice 0.0012 ...,dexitPriceBot2,deXit Price Bot 🤖,dexitPriceBot,deXit community,False,This is a bot for deXit blockchain shows the p...
3,2022-11-24 09:19:05+00:00,@cryptojack @AltGemsAlert My biggest #crypto b...,Twitter Web App,Stacey Mitchell,StablePump,NaN,False,"#web3 entrepreneur, dreamer, lifelong learner"
4,2022-11-24 09:19:05+00:00,What Do You Really Own in the Ownership Econom...,Agorapulse app,Richard Turrin,richardturrin,Shanghai,False,Author best seller: Cashless | Onalytica No. 4...


In [5]:
def cleaning_tweet_data(df: pd.DataFrame()):
    df_tweets = df.copy()
    df_tweets.dropna(inplace=True)
    if 'Unnamed: 0' in df_tweets.columns:
        df_tweets.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
    df_tweets.drop(columns=['source', 'name', 'location', 'verified', 'description'], axis=1, inplace=True)
    blanks = []  # start with an empty list

    for i, created_at, text, username in df_tweets.itertuples():  
        if type(text)==str:            
            if text.isspace():         
                blanks.append(i)    

    df_tweets.drop(blanks, inplace=True)
    df_tweets = df_tweets.apply(lambda x: x.astype(str).str.lower()).drop_duplicates(subset=['text', 'username'], keep='first')

    df_tweets['text'] = df_tweets['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
    df_tweets['text'] = df_tweets['text'].str.lower()
    df_tweets['text'] = df_tweets['text'].str.replace("@[a-z0-9A-Z]+", "", regex=True)
    df_tweets['text'] = df_tweets['text'].str.replace("#[a-z0-9A-Z]+","", regex=True)
    df_tweets['text'] = df_tweets['text'].str.replace(r"http\S+", "")
    df_tweets['text'] = df_tweets['text'].str.replace(r"www.\S+", "")
    df_tweets['text'] = df_tweets['text'].str.replace('[()!?]', ' ')
    df_tweets['text'] = df_tweets['text'].str.replace('\[.*?\]',' ')
    df_tweets['text'] = df_tweets['text'].str.replace("[^a-z0-9]"," ")
    df_tweets['words'] = df_tweets['text'].apply(lambda x:str(x.lower()).split())

    df_tweets['tweet_without_stopwords'] = df_tweets['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))

    def remove_stopword(x):
        return [y for y in x if y not in stopwords.words('english')]
    df_tweets['words'] = df_tweets['words'].apply(lambda x:remove_stopword(x))

    def remove_hashtag(x):
        return [y for y in x if not y.startswith('#')]
    df_tweets['words'] = df_tweets['words'].apply(lambda x:remove_hashtag(x))

    return df_tweets

In [6]:
df_btc = cleaning_tweet_data(df_btc)